In [1]:
import pickle
import numpy as np, os, os.path, sys, warnings
import pandas as pd

from keras.layers import Dense, Dropout, Embedding
from keras.models import Input, Model
from keras.preprocessing import sequence
from tensorflow.python.framework import ops
from tcn import TCN
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras import callbacks
from sklearn.preprocessing import normalize
from matplotlib import pyplot as plt
from keras.models import load_model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Using TensorFlow backend.


In [2]:
def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
      # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

In [3]:
def evaluate_sepsis_score(labels, prediction):

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0
    num_files= len(labels)
    

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        num_rows          = len(labels[k])
        observed_predictions = prediction[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels[k]):
            t_sepsis = np.argmax(labels[k]) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels[k], observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels[k], best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels[k], worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels[k], inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return  normalized_observed_utility

In [4]:
x=pickle.load(open( "x_miss_ff", "rb" ))
y=pickle.load(open( "y", "rb" ))

In [36]:
def get_weights(pt):
    s_weights=np.ones(len(pt))*.05
    if pt[-1]==1:   
        t_sepsis=np.where(pt==1)[0][0]
        s_weights[t_sepsis]=1*10
        for j in range(t_sepsis-6,t_sepsis):
            s_weights[j]=-1*np.absolute(1-1/6*(t_sepsis-j))*10
        for k in range(t_sepsis+1,len(pt)):
            pos=1-(k-t_sepsis)*1/9
            neg=-(k-t_sepsis)*2/9
            s_weights[k]=(pos-neg)*10
    return s_weights

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1,random_state=42)

In [7]:
for i in range(len(x_train)):
    x_train[i]=np.asarray(x_train[i])
    y_train[i]=np.asarray(y_train[i])

In [8]:
for i in range(len(x_test)):
    x_test[i]=np.asarray(x_test[i])
    y_test[i]=np.asarray(y_test[i])

In [37]:
def get_x_y(x_train,y_train):
    while True:
        for k in range(len(x_train)):
            x_t = x_train[k]
            y_t = y_train[k]
            sample_weights=get_weights(y_train[k])
            sample_weights=sample_weights.reshape(len(sample_weights),1)
            yield (np.expand_dims(np.array(x_t),axis=0), np.expand_dims(np.array(y_t),axis=0),sample_weights.T)

In [10]:
i = Input(batch_shape=(1, None, 69))

o = TCN(nb_filters=30, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4,8,16,32], padding='causal', use_skip_connections=True, dropout_rate=0.5, return_sequences=True, name='tcn')(i)
o = Dense(1,activation='sigmoid')(o)

model = Model(inputs=[i], outputs=[o])

In [38]:
model.compile(optimizer='sgd',loss='mse', metrics=['binary_accuracy','mse','binary_crossentropy'],sample_weight_mode='temporal')

In [39]:
es = callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10,restore_best_weights=True)

In [40]:
gen = get_x_y(x_train=x_train,y_train=y_train)

In [ ]:
model.fit_generator(gen, epochs=300, steps_per_epoch=len(x_train), verbose=1,callbacks=[es],shuffle=True)

Epoch 1/300
36302/36302 [==============================] - 390s 11ms/step - loss: -0.0015 - binary_accuracy: 0.1586 - mean_squared_error: 0.6991 - binary_crossentropy: 3.5075
Epoch 2/300
36302/36302 [==============================] - 391s 11ms/step - loss: -0.0026 - binary_accuracy: 0.1720 - mean_squared_error: 0.6922 - binary_crossentropy: 3.7169
Epoch 3/300
 8971/36302 [======>.......................] - ETA: 4:53 - loss: -0.0044 - binary_accuracy: 0.2058 - mean_squared_error: 0.6662 - binary_crossentropy: 3.6672

In [ ]:
model.save('TCN_june10_365_trial2_1.h5')  

In [ ]:
def get_weights(pt):
    s_weights=np.ones(len(pt))*.05
    if pt[-1]==1:   
        t_sepsis=np.where(pt==1)[0][0]
        s_weights[t_sepsis]=1*5
        for j in range(t_sepsis-6,t_sepsis):
            s_weights[j]=-1*np.absolute(1-1/6*(t_sepsis-j))*5
        for k in range(t_sepsis+1,len(pt)):
            pos=1-(k-t_sepsis)*1/9
            neg=-(k-t_sepsis)*2/9
            s_weights[k]=(pos-neg)*5
    return s_weights

In [ ]:
gen = get_x_y(x_train=x_train,y_train=y_train)
model.fit_generator(gen, epochs=300, steps_per_epoch=len(x_train), verbose=1,callbacks=[es],shuffle=True)
model.save('TCN_june10_365_trial2_1_2_v2.h5')  

In [ ]:
model.fit_generator(gen, epochs=300, steps_per_epoch=len(x_train), verbose=1,callbacks=[es],shuffle=True)

Epoch 1/300
36302/36302 [==============================] - 401s 11ms/step - loss: 0.0228 - binary_accuracy: 0.5660 - mean_squared_error: 0.2724 - binary_crossentropy: 0.8191
Epoch 2/300
36302/36302 [==============================] - 392s 11ms/step - loss: 0.0188 - binary_accuracy: 0.6590 - mean_squared_error: 0.2331 - binary_crossentropy: 0.6915
Epoch 3/300
36302/36302 [==============================] - 393s 11ms/step - loss: 0.0179 - binary_accuracy: 0.6845 - mean_squared_error: 0.2250 - binary_crossentropy: 0.6855
Epoch 4/300
36302/36302 [==============================] - 392s 11ms/step - loss: 0.0175 - binary_accuracy: 0.7010 - mean_squared_error: 0.2212 - binary_crossentropy: 0.6913
Epoch 5/300
36302/36302 [==============================] - 392s 11ms/step - loss: 0.0170 - binary_accuracy: 0.7106 - mean_squared_error: 0.2170 - binary_crossentropy: 0.6883
Epoch 6/300
36302/36302 [==============================] - 392s 11ms/step - loss: 0.0167 - binary_accuracy: 0.7189 - mean_squared_

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



36302/36302 [==============================] - 402s 11ms/step - loss: 0.0153 - binary_accuracy: 0.7413 - mean_squared_error: 0.2014 - binary_crossentropy: 0.6946
Epoch 16/300
13321/36302 [==========>...................] - ETA: 4:05 - loss: 0.0152 - binary_accuracy: 0.7296 - mean_squared_error: 0.2062 - binary_crossentropy: 0.7077

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



36302/36302 [==============================] - 390s 11ms/step - loss: 0.0152 - binary_accuracy: 0.7379 - mean_squared_error: 0.2020 - binary_crossentropy: 0.6973
Epoch 17/300
36302/36302 [==============================] - 391s 11ms/step - loss: 0.0150 - binary_accuracy: 0.7442 - mean_squared_error: 0.2000 - binary_crossentropy: 0.6931
Epoch 18/300
36302/36302 [==============================] - 391s 11ms/step - loss: 0.0150 - binary_accuracy: 0.7465 - mean_squared_error: 0.1976 - binary_crossentropy: 0.6860
Epoch 19/300
36302/36302 [==============================] - 392s 11ms/step - loss: 0.0148 - binary_accuracy: 0.7424 - mean_squared_error: 0.1994 - binary_crossentropy: 0.6909
Epoch 20/300
36302/36302 [==============================] - 391s 11ms/step - loss: 0.0147 - binary_accuracy: 0.7419 - mean_squared_error: 0.1978 - binary_crossentropy: 0.6871
Epoch 21/300
14750/36302 [===========>..................] - ETA: 3:53 - loss: 0.0149 - binary_accuracy: 0.7406 - mean_squared_error: 0.199

In [ ]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test[:2000]))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.4)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

In [35]:
model.save('TCN_june10_365_1.h5')  

In [34]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.45)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.25936919237508627

In [33]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.55)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.33362448892900753

In [32]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.6)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.34955264694950355

In [29]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.65)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.3575585408591303

In [28]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.7)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.36150639834333353

In [30]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.75)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.34594594594594597

In [31]:
label=list()
prediction=list()
probability=list()
for i in range(len((x_test))):
    probability.append((model.predict(np.expand_dims(x_test[i],axis=0))[0]))
    prediction.append(((model.predict(np.expand_dims(x_test[i],axis=0))[0])>0.8)*1)
    label.append(np.asarray(y_test[i]))
evaluate_sepsis_score(labels=label, prediction=prediction)

0.33459618754314235